## **Criando Métricas Personalizadas Para Keras/Tensorflow**

Você pode implementar sua própria métrica e usá-la como objetivo e na busca de hiperparâmetros.


Nesse notebook, iremos implementar com o Keras/Tensorflow duas métricas do Scikit-Learn para avaliar o desempenho de uma Rede Neural Convolucional (CNN):
* F1-Score
* BalancedAccuracy

[Custom metric as the objective](https://keras.io/guides/keras_tuner/getting_started/#tune-model-training)

#### **Importando as Bibliotecas**

In [165]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.16.1


#### **Classificação de Texto com Avaliações de Filmes**


Este notebook classifica avaliações de filmes como positiva ou negativa usando o texto da avaliação. Isto é um exemplo de classificação binária —ou duas-classes—, um importante e bastante aplicado tipo de problema de aprendizado de máquina.

Usaremos a base de dados IMDB que contém avaliaçòes de mais de 50000 filmes da base de dados Internet Movie Database. 

A base é dividida em 25000 avaliações para treinamento e 25000 para teste. Os conjuntos de **treinamentos e testes são balanceados**, ou seja, eles possuem a mesma quantidade de avaliações positivas e negativas.

O notebook utiliza tf.keras, uma API alto-nível para construir e treinar modelos com TensorFlow

[Text Classification Keras/Tensorflow](https://www.tensorflow.org/tutorials/keras/text_classification?hl=pt-br)

**Baixe a base de dados IMDB**

In [166]:
# A base de dados vem empacotada com TensorFlow. 
# Ela já vem pré-processada de forma que as avaliações (sequências de palavras) foram convertidas em sequências de inteiros, onde cada inteiro representa uma palavra específica no dicionário.
# O argumento num_words=10000 mantém as 10000 palavras mais frequentes no conjunto de treinamento. As palavras mais raras são descartadas para preservar o tamanho dos dados de forma maleável.

imdb = tf.keras.datasets.imdb

**Conjunto de Treinamento e Teste**

In [167]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Training entries: 25000, labels: 25000


**Explore os dados**

In [168]:
# Vamos parar um momento para entender o formato dos dados. 
# O conjunto de dados vem pré-processado: cada exemplo é um array de inteiros representando as palavras da avaliação do filme. 
# Cada label é um inteiro com valor ou de 0 ou 1, onde 0 é uma avaliação negativa e 1 é uma avaliação positiva.
# O texto das avaliações foi convertido para inteiros, onde cada inteiro representa uma palavra específica no dicionário. Isso é como se parece a primeira revisão:

print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [169]:
# As avaliações dos filmes têm tamanhos diferentes. 
# O código abaixo mostra o número de palavras da primeira e segunda avaliação. 
# Sabendo que o número de entradas da rede neural tem que ser o mesmo também, temos que resolver isto mais tarde.

len(train_data[0]), len(train_data[1])

(218, 189)

In [170]:
train_data

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

In [171]:
train_labels

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

**Prepare os Dados**

As avaliações —os arrays de inteiros— devem ser convertidas em tensores (tensors) antes de alimentar a rede neural. Essa conversão pode ser feita de duas formas:

* Converter os arrays em vetores de 0s e 1s indicando a ocorrência da palavra, similar com one-hot encoding. Por exemplo, a sequência [3, 5] se tornaria um vetor de 10000 dimensões, onde todos seriam 0s, tirando 3 e 5, que são 1s. Depois, faça disso a primeira camada da nossa rede neural — a Dense layer — que pode trabalhar com dados em ponto flutuante. Essa abordagem é intensa em relação a memória, logo requer uma matriz de tamanho num_words * num_reviews.

* Alternativamente, podemos preencher o array para que todos tenho o mesmo comprimento, e depois criar um tensor inteiro de formato max_length * num_reviews. Podemos usar uma camada embedding capaz de lidar com o formato como a primeira camada da nossa rede.

Nesse notebook, usaremos a segunda abordagem.

In [172]:
# Já que as avaliações dos filmes devem ter o mesmo tamanho, usaremos a função pad_sequences para padronizar os tamanhos:

# Um dicionário mapeando palavras em índices inteiros
word_index = imdb.get_word_index()
word_index["<PAD>"] = 0

train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [173]:
train_data

array([[   1,   14,   22, ...,    0,    0,    0],
       [   1,  194, 1153, ...,    0,    0,    0],
       [   1,   14,   47, ...,    0,    0,    0],
       ...,
       [   1,   11,    6, ...,    0,    0,    0],
       [   1, 1446, 7079, ...,    0,    0,    0],
       [   1,   17,    6, ...,    0,    0,    0]])

In [174]:
# Agora, vamos olhar o tamanho dos exemplos:


len(train_data[0]), len(train_data[1])

(256, 256)

**Criando um Conjunto de Validação**

Quando treinamos. queremos checar a acurácia do modelo com os dados que ele nunca viu. 

Crie uma conjunto de validação tirando 10000 exemplos do conjunto de treinamento original. 

(Por que não usar o de teste agora? Nosso objetivo é desenvolver e melhorar (tunar) nosso modelo usando somente os dados de treinamento, depois usaremos o teste uma única vez para avaliar a previsão).

In [175]:
X_valid = train_data[:10000]
X_train = train_data[10000:]

y_valid = train_labels[:10000]
y_train = train_labels[10000:]

**Criando a Métrica F1-Score Customizada (sem parâmetro) usando Keras**

[f1-score-guide](https://www.v7labs.com/blog/f1-score-guide)

In [176]:
# O Tensorflow 2 possui uma opção de configuração para executar funções "avidamente", o que permitirá obter valores do Tensor por meio do método .numpy(). 
# Para habilitar a execução antecipada, use o seguinte comando: tf.config.run_functions_eagerly(True)
tf.config.run_functions_eagerly(True)

tf.data.experimental.enable_debug_mode()

In [177]:
class CustomF1ScoreA(tf.keras.metrics.Metric):
    def __init__(self, name='f1-score-macro', **kwargs):
        super().__init__(name=name, **kwargs)
        self.f1_macro = self.add_variable(shape=(), initializer='zeros', name='f1-macro')

    def update_state(self, y_true, y_pred, sample_weight=None):
        
        y_pred = tf.cast(tf.reshape(tf.round(y_pred), -1), 'int32')
        #y_true = tf.cast(tf.reshape(y_true, shape=(-1, 1)), 'int32')

        y_true = tf.keras.ops.cast(y_true, "bool")
        y_pred = tf.keras.ops.cast(y_pred, "bool")

        # Verifica a quantidade de VERDADEIROS POSITIVOS (TP)
        values = tf.keras.ops.logical_and(tf.keras.ops.equal(y_true, True), tf.keras.ops.equal(y_pred, True))
        values = tf.keras.ops.cast(values, dtype='int32')
        tp = tf.keras.ops.sum(values).numpy()
        
        # Verifica a quantidade de VERDADEIROS NEGATIVOS (TN)
        values = tf.keras.ops.logical_and(tf.keras.ops.equal(y_true, False), tf.keras.ops.equal(y_pred, False))
        values = tf.keras.ops.cast(values, dtype='int32')
        tn = tf.keras.ops.sum(values).numpy()
        
        # Verifica a quantidade de FALSO POSITIVO (FP)
        values = tf.keras.ops.logical_and(tf.keras.ops.equal(y_true, False), tf.keras.ops.equal(y_pred, True))
        values = tf.keras.ops.cast(values, dtype='int32')
        fp = tf.keras.ops.sum(values).numpy()

        # Verifica a quantidade de FALSO NEGATIVO (FN)
        values = tf.keras.ops.logical_and(tf.keras.ops.equal(y_true, True), tf.keras.ops.equal(y_pred, False))
        values = tf.keras.ops.cast(values, dtype='int32')
        fn = tf.keras.ops.sum(values).numpy()

        f1_1 = (2 * tp) / (2 * tp + fp + fn)
        f1_0 = (2 * tn) / (2 * tn + fp + fn)
        f1 = (f1_1 + f1_0) / 2

        self.f1_macro.assign(f1)

    def result(self):
        return self.f1_macro

    def reset_states(self):
        self.f1_macro.assign(0)

**Criando a Métrica F1-Score Customizada (com parâmetro) usando Keras**

In [178]:
class CustomF1ScoreB(tf.keras.metrics.Metric): 
    def __init__(self, name='f1', Tipo='macro'):
        super(CustomF1ScoreB, self).__init__()
        self.f1 = self.add_weight(name=name, initializer='zeros')
        self.Tipo = Tipo
    
    def update_state(self, y_true, y_pred, sample_weight=None):
    
        y_pred = tf.cast(tf.reshape(tf.round(y_pred), -1), 'int32')
        #y_true = tf.cast(tf.reshape(y_true, shape=(-1, 1)), 'int32')

        y_true = tf.keras.ops.cast(y_true, "bool")
        y_pred = tf.keras.ops.cast(y_pred, "bool")

        # Verifica a quantidade de VERDADEIROS POSITIVOS (TP)
        values = tf.keras.ops.logical_and(tf.keras.ops.equal(y_true, True), tf.keras.ops.equal(y_pred, True))
        values = tf.keras.ops.cast(values, dtype='int32')
        tp = tf.keras.ops.sum(values).numpy()
        
        # Verifica a quantidade de VERDADEIROS NEGATIVOS (TN)
        values = tf.keras.ops.logical_and(tf.keras.ops.equal(y_true, False), tf.keras.ops.equal(y_pred, False))
        values = tf.keras.ops.cast(values, dtype='int32')
        tn = tf.keras.ops.sum(values).numpy()
        
        # Verifica a quantidade de FALSO POSITIVO (FP)
        values = tf.keras.ops.logical_and(tf.keras.ops.equal(y_true, False), tf.keras.ops.equal(y_pred, True))
        values = tf.keras.ops.cast(values, dtype='int32')
        fp = tf.keras.ops.sum(values).numpy()

        # Verifica a quantidade de FALSO NEGATIVO (FN)
        values = tf.keras.ops.logical_and(tf.keras.ops.equal(y_true, True), tf.keras.ops.equal(y_pred, False))
        values = tf.keras.ops.cast(values, dtype='int32')
        fn = tf.keras.ops.sum(values).numpy()

        f1_1 = (2 * tp) / (2 * tp + fp + fn)
        f1_0 = (2 * tn) / (2 * tn + fp + fn)

        f = 0

        if self.Tipo == 'macro':
            f = (f1_1 + f1_0) / 2
        elif self.Tipo == 'micro':
            f = (tp + tn) / (tp + fp + fn + tn)
        elif self.Tipo == 'weighted':
            f = (((tp+fn) * f1_1) + ((fp + tn) * f1_0)) / ((tp+fn) + (fp + tn))
            
        self.f1.assign(f)

    def result(self):
        return self.f1

    def reset_states(self):
        self.f1.assign(0)

**Criando a Métrica F1-Score Customizada (com parâmetro) usando Keras e Numpy**

In [179]:
class CustomF1ScoreN(tf.keras.metrics.Metric): 
    def __init__(self, name='f1', Tipo='macro'):
        super(CustomF1ScoreN, self).__init__()
        self.f1 = self.add_weight(name=name, initializer='zeros')
        self.Tipo = Tipo
    
    def update_state(self, y_true, y_pred, sample_weight=None):
    
        y_pred = np.reshape(np.round(y_pred), -1)
        #y_true = np.reshape(y_true, shape=(-1, 1))

        # Transforma o array/list de inteiros para booleano: [0,1,0,1] --> [False,  True, False,  True]
        y_true = np.bool_(y_true)
        y_pred = np.bool_(y_pred)

        # Verifica a quantidade de VERDADEIROS POSITIVOS (TP)
        values = np.logical_and(np.equal(y_true, True), np.equal(y_pred, True))
        # Transforma o array/list de booleano para inteiro: [False,  True, False,  True] --> [0,1,0,1]
        values = values.astype(int)  
        # Faz a soma do array/list
        tp = np.sum(values)
        
        # Verifica a quantidade de VERDADEIROS NEGATIVOS (TN)
        values = np.logical_and(np.equal(y_true, False), np.equal(y_pred, False))
        # Transforma o array/list de booleano para inteiro: [False,  True, False,  True] --> [0,1,0,1]
        values = values.astype(int)  
        # Faz a soma do array/list
        tn = np.sum(values)

        # Verifica a quantidade de FALSO POSITIVO (FP)
        values = np.logical_and(np.equal(y_true, False), np.equal(y_pred, True))
        # Transforma o array/list de booleano para inteiro: [False,  True, False,  True] --> [0,1,0,1]
        values = values.astype(int)  
        # Faz a soma do array/list
        fp = np.sum(values)

        # Verifica a quantidade de FALSO NEGATIVO (FN)
        values = np.logical_and(np.equal(y_true, True), np.equal(y_pred, False))
        # Transforma o array/list de booleano para inteiro: [False,  True, False,  True] --> [0,1,0,1]
        values = values.astype(int)  
        # Faz a soma do array/list
        fn = np.sum(values)

        f1_1 = (2 * tp) / (2 * tp + fp + fn)
        f1_0 = (2 * tn) / (2 * tn + fp + fn)

        
        # cf_matrix = confusion_matrix(y_true, y_pred)
        # tn, fp, fn, tp = cf_matrix.ravel()

        f = 0

        if self.Tipo == 'macro':
            f = (f1_1 + f1_0) / 2
        elif self.Tipo == 'micro':
            f = (tp + tn) / (tp + fp + fn + tn)
        elif self.Tipo == 'weighted':
            f = (((tp+fn) * f1_1) + ((fp + tn) * f1_0)) / ((tp+fn) + (fp + tn))
            
        self.f1.assign(f)

    def result(self):
        return self.f1

    def reset_states(self):
        self.f1.assign(0)

**Métrica F1-Score Customizada (sem parâmetros) usando Keras e Scikit-Learn**

In [180]:
from sklearn.metrics import f1_score

class CustomF1Score_SP(tf.keras.metrics.Metric):
    def __init__(self, name='f1-score-macro', **kwargs):
        super().__init__(name=name, **kwargs)
        self.f1_macro = self.add_weight(name='f1-macro', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        
        y_pred_ = np.array(np.reshape(np.round(y_pred), -1), np.int32)

        f1 = np.round(f1_score(y_true, y_pred_, average="macro"), 2)

        self.f1_macro.assign(f1)     

    def result(self):
        return self.f1_macro

    def reset_states(self):
        self.f1_macro.assign(0)

**Métrica Balanced Accuracy Customizada (sem parâmetros) usando Keras e Scikit-Learn**

A balanced_accuracy_score função calcula a precisão balanceada, o que evita estimativas de desempenho inflacionadas em conjuntos de dados desequilibrados. 

No caso binário, a precisão balanceada é igual à média aritmética de sensibilidade (taxa verdadeira positiva) e especificidade (taxa verdadeira negativa), ou a área sob a curva ROC com previsões binárias em vez de pontuações.

In [181]:
from sklearn.metrics import balanced_accuracy_score

class CustomBalancedAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='CustomBalancedAccuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.balanced_acc = self.add_weight(name='Acc', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        
        y_pred_ = np.array(np.reshape(np.round(y_pred), -1), np.int32)

        bc = np.round(balanced_accuracy_score(y_true, y_pred_), 2) 

        self.balanced_acc.assign(bc)     

    def result(self):
        return self.balanced_acc

    def reset_states(self):
        self.balanced_acc.assign(0)

**Métrica Balanced Accuracy Customizada (sem parâmetros) usando Keras**

In [182]:
class CustomBalancedAccuracyC(tf.keras.metrics.Metric):
    def __init__(self, name='CustomBalancedAccuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.balanced_acc = self.add_weight(name='Acc', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
    
        y_pred = np.reshape(np.round(y_pred), -1)
        #y_true = np.reshape(y_true, shape=(-1, 1))

        # Transforma o array/list de inteiros para booleano: [0,1,0,1] --> [False,  True, False,  True]
        y_true = np.bool_(y_true)
        y_pred = np.bool_(y_pred)

        # Verifica a quantidade de VERDADEIROS POSITIVOS (TP)
        values = np.logical_and(np.equal(y_true, True), np.equal(y_pred, True))
        # Transforma o array/list de booleano para inteiro: [False,  True, False,  True] --> [0,1,0,1]
        values = values.astype(int)  
        # Faz a soma do array/list
        tp = np.sum(values)
        
        # Verifica a quantidade de VERDADEIROS NEGATIVOS (TN)
        values = np.logical_and(np.equal(y_true, False), np.equal(y_pred, False))
        # Transforma o array/list de booleano para inteiro: [False,  True, False,  True] --> [0,1,0,1]
        values = values.astype(int)  
        # Faz a soma do array/list
        tn = np.sum(values)

        # Verifica a quantidade de FALSO POSITIVO (FP)
        values = np.logical_and(np.equal(y_true, False), np.equal(y_pred, True))
        # Transforma o array/list de booleano para inteiro: [False,  True, False,  True] --> [0,1,0,1]
        values = values.astype(int)  
        # Faz a soma do array/list
        fp = np.sum(values)

        # Verifica a quantidade de FALSO NEGATIVO (FN)
        values = np.logical_and(np.equal(y_true, True), np.equal(y_pred, False))
        # Transforma o array/list de booleano para inteiro: [False,  True, False,  True] --> [0,1,0,1]
        values = values.astype(int)  
        # Faz a soma do array/list
        fn = np.sum(values)

        f = 1/2 * ((tp/(tp+fn)) + (tn/(tn+fp)))
       
        
        # cf_matrix = confusion_matrix(y_true, y_pred)
        # tn, fp, fn, tp = cf_matrix.ravel()

        self.balanced_acc.assign(f)    

    def result(self):
        return self.balanced_acc

    def reset_states(self):
        self.balanced_acc.assign(0)

**Métrica Balanced Accuracy Customizada (com parâmetros) usando Keras**

In [183]:
# Aplicar limite de decisão para as probabilidades do modelo
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')


class CustomBalancedAccuracy_CP(tf.keras.metrics.Metric):
    def __init__(self, name='CustomBalancedAccuracy', Threshold=0):
        super(CustomBalancedAccuracy_CP, self).__init__()
        self.balanced_acc = self.add_weight(name='Acc', initializer='zeros')
        self.Threshold = Threshold

    def update_state(self, y_true, y_pred, sample_weight=None):
        
        y_pred_ = np.reshape(y_pred, -1)

        f1 = np.round(balanced_accuracy_score(y_true, to_labels(y_pred_, self.Threshold)), 2)
        
        self.balanced_acc.assign(f1)  

    def result(self):
        return self.balanced_acc

    def reset_states(self):
        self.balanced_acc.assign(0)

**Construindo o Modelo**

In [184]:
# O formato de entrada é a contagem vocabulário usados pelas avaliações dos filmes (10000 palavras)
vocab_size = 10000
max_length = 16

# Configurando a arquitetura do modelo  
model = tf.keras.Sequential()

# --> Camada de Entrada
# Geralmente, todas as camadas no Keras precisam conhecer a forma de suas entradas para poder criar seus pesos
# Neste caso, você deve iniciar seu modelo passando um objeto Input para seu modelo, para que ele conheça sua forma de entrada desde o início
# Modelos construídos com um formato de entrada predefinido como esse sempre possuem pesos (mesmo antes de ver qualquer dado) e sempre possuem um formato de saída definido.
# Em geral, é uma prática recomendada sempre especificar antecipadamente o formato de entrada de um modelo Sequencial, se você souber o que é.
# Uma alternativa simples é apenas passar um argumento input_shape para sua primeira camada:
# Exemplo: model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=max_length, input_shape=(max_length,)))
# No entanto, o exemplo acima gera um warning: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
# https://keras.io/guides/sequential_model/
# https://www.tensorflow.org/guide/keras/sequential_model?hl=pt-br
model.add(tf.keras.Input(shape=(max_length,)))

# Adicionando Camada de Incorporação - Transforma inteiros positivos (índices) em vetores densos de tamanho fixo.
model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=max_length))

# --> Camadas Intermediárias
# Adicionando Camada de Convolução
model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=5, activation='relu'))

# Adicionando Camada de Pooling
model.add(tf.keras.layers.GlobalMaxPooling1D())
                            
# Adicionando Camada Totalmente Conectada
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.30))

# --> Camada de Saída                 
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Sumário do modelo
#model.summary()

# Configurando o modelo para treinamento
model.compile(optimizer=tf.keras.optimizers.Adam(name='adam', learning_rate=0.001),   # O learning_rate padrão é 0.001 
                # Já que é um problema de classificação binário e o modelo tem como saída uma probabilidade usaremos a função loss "binary_crossentropy". A binary_crossentropy é a melhor função de loss para tratar probabilidades— ela mede a "distância" entre as distribuições de probabilidade, ou, no nosso caso, sobre a distribuição real e as previsões.
                loss=tf.keras.losses.BinaryCrossentropy(name='binary_crossentropy'), 
                # Usaremos como métrica a binary_accuracy. Essa métrica é utilizada especificamente em problemas de classificação binária e calcula com que frequência as previsões correspondem aos rótulos binários.
                metrics=[CustomF1ScoreA()])

In [185]:
# Treinando o modelo
history = model.fit(x=X_train,
                    y=y_train,
                    epochs=2,
                    batch_size=512,
                    verbose=1,
                    validation_data=(X_valid, y_valid))

Epoch 1/2


30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 321ms/step - f1-score-macro: 0.4821 - loss: 0.6926 - val_f1-score-macro: 0.7165 - val_loss: 0.6836
Epoch 2/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 352ms/step - f1-score-macro: 0.7049 - loss: 0.6575 - val_f1-score-macro: 0.7469 - val_loss: 0.5494


In [186]:
m = CustomF1ScoreA()
m.update_state(test_labels, model.predict(test_data))
f1_1 = tf.keras.ops.round(tf.convert_to_tensor(m.result().numpy(), dtype=tf.float64), 2)*100

print(f'{f1_1}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step
73.0


In [187]:
m = CustomF1ScoreB(Tipo='micro')
m.update_state(test_labels, model.predict(test_data))
f1_1 = tf.keras.ops.round(tf.convert_to_tensor(m.result().numpy(), dtype=tf.float64), 2)*100

m = CustomF1ScoreB(Tipo='macro')
m.update_state(test_labels, model.predict(test_data))
f1_2 = tf.keras.ops.round(tf.convert_to_tensor(m.result().numpy(), dtype=tf.float64), 2)*100

m = CustomF1ScoreB(Tipo='weighted')
m.update_state(test_labels, model.predict(test_data))
f1_3 = tf.keras.ops.round(tf.convert_to_tensor(m.result().numpy(), dtype=tf.float64), 2)*100

print(f'{f1_1} {f1_2} {f1_3}')

y_pred_ = np.array(np.reshape(np.round(model.predict(test_data)), -1), np.int32)
c_test_1 = np.round(f1_score(test_labels, y_pred_, average='micro'), 2)

y_pred_ = np.array(np.reshape(np.round(model.predict(test_data)), -1), np.int32)
c_test_2 = np.round(f1_score(test_labels, y_pred_, average='macro'), 2)

y_pred_ = np.array(np.reshape(np.round(model.predict(test_data)), -1), np.int32)
c_test_3 = np.round(f1_score(test_labels, y_pred_, average='weighted'), 2)

print(f'{c_test_1} {c_test_2} {c_test_3}')


782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step
74.0 73.0 73.0
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step
0.74 0.73 0.73


In [188]:
m = CustomF1ScoreN(Tipo='micro')
m.update_state(test_labels, model.predict(test_data))
f1_1 = tf.keras.ops.round(tf.convert_to_tensor(m.result().numpy(), dtype=tf.float64), 2)*100

m = CustomF1ScoreN(Tipo='macro')
m.update_state(test_labels, model.predict(test_data))
f1_2 = tf.keras.ops.round(tf.convert_to_tensor(m.result().numpy(), dtype=tf.float64), 2)*100

m = CustomF1ScoreN(Tipo='weighted')
m.update_state(test_labels, model.predict(test_data))
f1_3 = tf.keras.ops.round(tf.convert_to_tensor(m.result().numpy(), dtype=tf.float64), 2)*100

print(f'{f1_1} {f1_2} {f1_3}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step
74.0 73.0 73.0


In [189]:
m = CustomF1Score_SP()
m.update_state(test_labels, model.predict(test_data))
f1_0 = tf.keras.ops.round(tf.convert_to_tensor(m.result().numpy(), dtype=tf.float64), 2)*100

print(f'{f1_0}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step
73.0


In [190]:
c = CustomBalancedAccuracy()
c.update_state(test_labels, model.predict(test_data))
c_test_1 = tf.keras.ops.round(tf.convert_to_tensor(c.result().numpy(), dtype=tf.float64), 2)*100

c = CustomBalancedAccuracyC()
c.update_state(test_labels, model.predict(test_data))
c_test_2 = tf.keras.ops.round(tf.convert_to_tensor(c.result().numpy(), dtype=tf.float64), 2)*100

c = CustomBalancedAccuracy_CP(Threshold=0.356)
c.update_state(test_labels, model.predict(test_data))
c_test_3 = tf.keras.ops.round(tf.convert_to_tensor(c.result().numpy(), dtype=tf.float64), 2)*100

y_pred_ = np.array(np.reshape(np.round(model.predict(test_data)), -1), np.int32)
c_test_4 = np.round(balanced_accuracy_score(test_labels, y_pred_), 2)

print(f'{c_test_1} {c_test_2} {c_test_3} {c_test_4}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step
74.0 74.0 67.0 0.74
